In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
import pandas as pd
import os
from datetime import datetime

In [ ]:
#Define Variable
SCHEMA="main"
TABLE="survival"

In [ ]:
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['HADOOP_USER_NAME']='hive'
os.environ['PYSPARK_PYTHON'] ='/root/anaconda3/bin/python'
conf = pyspark.SparkConf().setAll([
     ('spark.driver.maxResultSize', '0'),
     ('spark.driver.memory', '2g'),
     ('spark.sql.repl.eagerEval.enabled','true'),
     ('hive.strict.managed.tables','false'),
     ('hive.metastore.uris', 'thrift://hive-metastore:9083'),
     ('metastore.client.capability.check','false')
    ])
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("street") \
        .config(conf=conf) \
        .enableHiveSupport() \
        .getOrCreate()

In [ ]:
df = pd.read_csv(f"https://vincentarelbundock.github.io/Rdatasets/csv/{TABLE}/nafld2.csv")
df = df.rename(columns={"Unnamed: 0": "idx"})

In [ ]:
df2 = spark.createDataFrame(df)
df2 = df2.sort(["idx"],ascending = True)

In [ ]:
if spark.sql(f"SHOW TABLES IN {SCHEMA}").filter(f"tableName == '{TABLE}'").count() > 0:
    repartitioned = df2.repartition('test')
    repartitioned.write \
		.mode('append') \
		.partitionBy('test') \
		.bucketBy(5, 'value') \
		.saveAsTable(f'{SCHEMA}.{TABLE}')
else:
    df2.write \
		.mode('overwrite') \
		.partitionBy('test') \
		.bucketBy(5, 'value') \
		.saveAsTable(f'{SCHEMA}.{TABLE}')